In [1]:
from qiskit.dagcircuit import DAGCircuit
from qiskit.circuit import QuantumCircuit, QuantumRegister, Gate,ParameterVector
from qiskit.circuit.library import CXGate, ECRGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.quantum_info import Operator, pauli_basis
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime.fake_provider import FakeMumbaiV2
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

import numpy as np
from typing import Iterable, Optional

In [2]:
class PauliTwirl(TransformationPass):
    """Add Pauli twirls to two-qubit gates."""
 
    def __init__(
        self,
        gates_to_twirl: Optional[Iterable[Gate]] = None,
    ):
        """
        Args:
            gates_to_twirl: Names of gates to twirl. The default behavior is to twirl all
                two-qubit basis gates, `cx` and `ecr` for IBM backends.
        """
        if gates_to_twirl is None:
            gates_to_twirl = [CXGate(), ECRGate()]
        self.gates_to_twirl = gates_to_twirl
        self.build_twirl_set()
        super().__init__()
 
    def build_twirl_set(self):
        """
        Build a set of Paulis to twirl for each gate and store internally as .twirl_set.
        """
        self.twirl_set = {}
 
        # iterate through gates to be twirled
        for twirl_gate in self.gates_to_twirl:
            twirl_list = []
 
            # iterate through Paulis on left of gate to twirl
            for pauli_left in pauli_basis(2):
                # iterate through Paulis on right of gate to twirl
                for pauli_right in pauli_basis(2):
                    # save pairs that produce identical operation as gate to twirl
                    if (Operator(pauli_left) @ Operator(twirl_gate)).equiv(
                        Operator(twirl_gate) @ pauli_right
                    ):
                        twirl_list.append((pauli_left, pauli_right))
 
            self.twirl_set[twirl_gate.name] = twirl_list
 
    def run(
        self,
        dag: DAGCircuit,
    ) -> DAGCircuit:
        # collect all nodes in DAG and proceed if it is to be twirled
        twirling_gate_classes = tuple(
            gate.base_class for gate in self.gates_to_twirl
        )
        for node in dag.op_nodes():
            if not isinstance(node.op, twirling_gate_classes):
                continue
 
            # random integer to select Pauli twirl pair
            pauli_index = np.random.randint(
                0, len(self.twirl_set[node.op.name])
            )
            twirl_pair = self.twirl_set[node.op.name][pauli_index]
 
            # instantiate mini_dag and attach quantum register
            mini_dag = DAGCircuit()
            register = QuantumRegister(2)
            mini_dag.add_qreg(register)
 
            # apply left Pauli, gate to twirl, and right Pauli to empty mini-DAG
            mini_dag.apply_operation_back(
                twirl_pair[0].to_instruction(), [register[0], register[1]]
            )
            mini_dag.apply_operation_back(node.op, [register[0], register[1]])
            mini_dag.apply_operation_back(
                twirl_pair[1].to_instruction(), [register[0], register[1]]
            )
 
            # substitute gate to twirl node with twirling mini-DAG
            dag.substitute_node_with_dag(node, mini_dag)
 
        return dag

In [3]:
def qiskit_circular_ansatz(N, reps=1, fix_2q=False): 
    qc = QuantumCircuit(N)

    # define your parameters
    p = ParameterVector('p', (N*2) *(reps+1)) 

    for r in range(reps): 
        for i in range(N):
            qc.ry(p[2*N*r+i], i)  
        for i in range(N):
            qc.rz(p[2*N*r+(i+N)], i)
        for i in range(N):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
    for i in range(N):
        qc.ry(p[2*N*reps+i], i)
    for i in range(N):
        qc.rz(p[2*N*reps + (i+N)], i)
    return qc

num_qubits = 10
qc = qiskit_circular_ansatz(num_qubits,reps=1)

In [4]:
np.random.seed(0)
paulis = ["".join(np.random.choice(['I', 'X', 'Y', 'Z'], size=num_qubits)) for _ in range(num_qubits)]
coeffs = np.random.random(len(paulis))
hamiltonian = SparsePauliOp.from_list(list(zip(paulis, coeffs)))

In [6]:
# from qiskit.quantum_info import SparsePauliOp
# from qiskit.circuit.library import EfficientSU2

# Define your Hamiltonian
# hamiltonian = SparsePauliOp.from_list([("Z", 1), ("X", 1)])

# Create an ansatz circuit
# num_qubits = hamiltonian.num_qubits
# ansatz = EfficientSU2(num_qubits, reps=2)

# Set initial parameters
import numpy as np
initial_params = np.random.random(qc.num_parameters)


In [7]:
from qiskit.transpiler import PassManager
from qiskit_aer.noise import NoiseModel

# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Create your custom pass
pm = PassManager([PauliTwirl()])

# Generate a preset pass manager
backend = FakeMumbaiV2() # Your quantum backend
noise_model = NoiseModel.from_backend(backend)

# Add your custom pass to the preset pass manager

In [16]:
from qiskit.primitives import Estimator

def vqe_cost_function(params, ansatz, hamiltonian, pass_manager, estimator):
    # Bind parameters to the ansatz
    bound_circuit = ansatz.assign_parameters(params)
    
    #Apply the pass manager
    twirled_qcs = [pass_manager.run(bound_circuit) for _ in range(50)]


    # Apply the pass manager
    # transpiled_circuit = pass_manager.run(bound_circuit)
    
    # Calculate expectation value
    energy_values = np.array([])
    for transpiled_circuit in twirled_qcs:
        job = estimator.run(transpiled_circuit, hamiltonian)
        result = job.result()
        energy = result.values[0]
        energy_values = np.append(energy_values, energy)
    
    # return result.values[0]
    return np.average(energy_values)


# Create an Estimator instance
estimator = Estimator()


In [19]:
from scipy.optimize import minimize

result = minimize(
    vqe_cost_function,
    initial_params,
    args=(qc, hamiltonian, pm, estimator),
    method='COBYLA',
    options={'maxiter': 5}
)

optimal_params = result.x
optimal_value = result.fun

print(f"Optimal value: {optimal_value}")
print(f"Optimal parameters: {optimal_params}")

Optimal value: -0.01348762901238022
Optimal parameters: [1.15896958 0.11037514 0.65632959 1.13818295 0.19658236 0.36872517
 0.82099323 0.09710128 0.83794491 0.09609841 0.97645947 0.4686512
 0.97676109 0.60484552 0.73926358 0.03918779 0.28280696 0.12019656
 0.2961402  0.11872772 0.31798318 0.41426299 0.0641475  0.69247212
 0.56660145 0.26538949 0.52324805 0.09394051 0.5759465  0.9292962
 0.31856895 0.66741038 0.13179786 0.7163272  0.28940609 0.18319136
 0.58651293 0.02010755 0.82894003 0.00469548]
